In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model

In [ ]:
# Carregar modelo treinado
model = load_model('mobilenetv2_emotion_finetuned_07-05.keras')

In [ ]:
# Rótulos das classes (ajuste conforme seu treinamento)
class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [ ]:
# Inicializar MTCNN
detector = MTCNN()

In [ ]:
# Tamanho de entrada do modelo
img_size = 96

In [ ]:
# Inicializar webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("Não foi possível acessar a webcam.")

print("Pressione 'q' para sair.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(rgb_frame)

    for result in results:
        x, y, w, h = result['box']
        x, y = abs(x), abs(y)  # Corrigir coordenadas negativas

        face = rgb_frame[y:y+h, x:x+w]
        if face.shape[0] < 20 or face.shape[1] < 20:
            continue  # Ignorar faces muito pequenas

        face_resized = cv2.resize(face, (img_size, img_size))
        face_input = np.expand_dims(face_resized, axis=0) / 255.0

        prediction = model.predict(face_input, verbose=0)
        predicted_class = np.argmax(prediction)
        predicted_label = class_labels[predicted_class]

        # Desenhar bounding box e label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, predicted_label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    cv2.imshow('Detecção de Emoções com MTCNN', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()